## **Action Net Model**

In [1]:
# # ENV: python==3.6.13 AND tensorflow==2.4.4
# # 
# # pip install tensorflow==2.4.4 && pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0 && pip install imageai==2.1.6
# # 
# !pip install tensorflow==2.4.4
# !pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0
# !pip install imageai==2.1.6
# 
# # !pip install imageai
# # 


In [2]:
from io import open
import requests
import shutil
from zipfile import ZipFile
from imageai.Prediction.Custom import ModelTraining, CustomImagePrediction
import os
# 

In [3]:
# 
execution_path = os.getcwd()
# 
SOURCE_PATH = "https://github.com/OlafenwaMoses/Action-Net/releases/download/v1/action_net_v1.zip"
FILE_DIR = os.path.join(execution_path, "action_net_v1.zip")
DATASET_DIR = os.path.join(execution_path, "action_net_v1.zip")
# 
# https://github.com/OlafenwaMoses/Action-Net/releases/download/v1/action_net_ex-060_acc-0.745313.h5
# https://huggingface.co/datasets/Bingsu/Human_Action_Recognition
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6631238/
# 

In [4]:
# if not exists('action_net_ex-060_acc-0.745313.h5'):
#   !wget https://raw.githubusercontent.com/OlafenwaMoses/Action-Net/master/model_class.json
#   !wget https://github.com/OlafenwaMoses/Action-Net/releases/download/v1/action_net_ex-060_acc-0.745313.h5
#   # 
#   !git clone https://github.com/OlafenwaMoses/Action-Net
#   !mkdir images && cp -r Action-Net/images ./images
#   # !unzip images.zip -d ./
# # 

In [5]:


def download_action_net():
    if (os.path.exists(FILE_DIR) == False):
        print("Downloading action_net_v1.zip")
        data = requests.get(SOURCE_PATH,
                            stream=True)

        with open(FILE_DIR, "wb") as file:
            shutil.copyfileobj(data.raw, file)
        del data

        extract = ZipFile(FILE_DIR)
        extract.extractall(execution_path)
        extract.close()


def train_action_net():
    download_action_net()

    trainer = ModelTraining()
    trainer.setModelTypeAsResNet()
    trainer.setDataDirectory("action_net_v1")
    trainer.trainModel(num_objects=16, num_experiments=200, batch_size=32, save_full_model=True,
                       enhance_data=True)

def run_predict(image_file_path: str = "images/5.jpg"):
    predictor = CustomImagePrediction()
    predictor.setModelPath(model_path="action_net_ex-060_acc-0.745313.h5")
    predictor.setJsonPath(model_json="model_class.json")
    predictor.loadFullModel(num_objects=16)


    # predictions, probabilities = predictor.predictImage(image_input=image_file_path, result_count=4)
    predictions, probabilities = predictor.classifyImage(image_input=image_file_path, result_count=4)
    # for prediction, probability in zip(predictions, probabilities):
    #     print(prediction, " : ", probability)

    return predictions, probabilities



In [6]:
#Un-comment the line below to train the model
#train_action_net()

#Un-comment the line below to run predictions
predictions, probabilities = run_predict("images/k3.jpg")
for prediction, probability in zip(predictions, probabilities):
    print(prediction, " : ", probability)
# 

kissing  :  68.00127625465393
laughing  :  15.769656002521515
calling  :  14.704068005084991
sleeping  :  1.4966920018196106


In [7]:
# https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/
# https://pyimagesearch.com/2019/07/15/video-classification-with-keras-and-deep-learning/
# 
# 
# Detect air sound from audio and video github
# 
# 

In [ ]:
import os
from typing import List
from io import BytesIO
from PIL import Image
from video2images import Video2Images
from application.main.config import settings
from application.main.utility.manager.image_utils import BasicImageUtils


class BasicVideoUtils(BasicImageUtils):

    @classmethod
    async def read_video_file(cls, file, filename, cache=True) -> List[Image.Image]:
        file_input_location = os.path.join(settings.APP_CONFIG.CACHE_DIR, f"inputs/{filename}")
        file_output_location = os.path.join(settings.APP_CONFIG.CACHE_DIR, "outputs")

        # Save video to a temporary file
        with open(file_input_location, 'wb') as f:
            f.write(file)
        
        # 
        try:
            # 
            frames_location = Video2Images(video_filepath=file_input_location, out_dir=file_output_location, capture_rate=10).out_dir
            # 
            images = []
            for filename in os.listdir(frames_location):
                if filename.endswith('.jpg'):
                    # send the file path
                    img_path = os.path.join(frames_location, filename)
                    images.append(img_path)
                    # 
                    # send the file bytes 
                    # image = cls.read_image_file(img_path, filename)
                    # images.append(image)
                    # with Image.open(BytesIO(img_path)) as img:
                    #     images.append(img)
        except:
            return []
        
        if cache:
            pass

        return images


In [ ]:
from typing import List, Dict
from collections import defaultdict
from pathlib import Path
from imageai.Prediction.Custom import CustomImagePrediction
# 
# from tensorflow.keras.applications.imagenet_utils import decode_predictions
from application.initializer import LoggerInstance
# from PIL import Image
import numpy as np
# import tensorflow as tf
import ssl
# 

ssl._create_default_https_context = ssl._create_unverified_context
logger = LoggerInstance().get_logger(__name__)
classifier_model = None


def merge_dicts(dicts_list):
    """
        To merge a list of dictionaries while taking the mean of values for repeated keys
    """
    merged_dict = defaultdict(lambda: {'sum': 0, 'count': 0})
    for d in dicts_list:
        for k, v in d.items():
            merged_dict[k]['sum'] += v
            merged_dict[k]['count'] += 1
    mean_dict = {k: v['sum'] / v['count'] for k, v in merged_dict.items()}
    return mean_dict


class InferenceTask:
    """
        Split video into frames, classify each frame, and ensemble the results
    """
    @staticmethod
    async def load_model(classifier_model_name):
        if len(classifier_model_name) != 0:
            model = CustomImagePrediction()
            model.setModelPath(model_path=classifier_model_name['model']) # "action_net_ex-060_acc-0.745313.h5"
            model.setJsonPath(model_json=classifier_model_name['classes']) # "model_class.json" https://raw.githubusercontent.com/OlafenwaMoses/Action-Net/master/model_class.json
            model.loadFullModel(num_objects=16)

        return model
    
    @staticmethod
    async def decode_results(results: List[zip]) -> List[Dict[str, float]]:
        # Dict[str, float]
        decoded_results = []
        # decoded_results = {}
        for result in results:
            _results = []
            for prediction, probability in result:
                resp = {}
                resp[prediction] = probability
                # resp["confidence"] = probability # f"{round(probability, 2):0.2f} %"
                _results.append(resp) 
            # 
            # print(_results)
            _merged_results = merge_dicts(_results)
            decoded_results.append(_merged_results)

        merged_results = merge_dicts(decoded_results)
        return merged_results
    
    async def predict(self, classifier_model_name, video: List[Path]) -> List:
        if len(video) is 0:
            return {"message": "Server failed to process file"}
        # 
        global classifier_model
        if classifier_model is None:
            classifier_model = await self.load_model(classifier_model_name)
        # 
        results = []
        # 
        for image in video:
            predictions, probabilities = classifier_model.classifyImage(image_input=image, result_count=5)
            results.append(zip(predictions, probabilities))

        response = await self.decode_results(results) # f"{round(probability, 2):0.2f} %"
        return response
# 
# 
# https://github.com/AbhishekSalian/Video2Images
# https://github.com/bhimrazy/Image-Recognition-App-using-FastAPI-and-PyTorch
# 
# 
# def run_predict(image_file_path: str = "images/5.jpg"):
#     predictor = CustomImagePrediction()
#     predictor.setModelPath(model_path="action_net_ex-060_acc-0.745313.h5")
#     predictor.setJsonPath(model_json="model_class.json")
#     predictor.loadFullModel(num_objects=16)


#     # predictions, probabilities = predictor.predictImage(image_input=image_file_path, result_count=4)
#     predictions, probabilities = predictor.classifyImage(image_input=image_file_path, result_count=4)
#     # for prediction, probability in zip(predictions, probabilities):
#     #     print(prediction, " : ", probability)

#     return predictions, probabilities
# 
# https://stackoverflow.com/questions/36758945/how-to-merge-a-list-of-dicts-summing-values-for-repeated-keys





In [ ]:
class VideoClassificationService(object):

    def __init__(self):
        self.logger = LoggerInstance().get_logger(__name__)
        self.video_model = InferenceTask()
        self.image_classification_model = settings.APP_CONFIG.ACTION_NET_IMAGE_CLASSIFICATION_MODEL
        self.image_classification_model_classes = settings.APP_CONFIG.ACTION_NET_IMAGE_CLASSIFICATION_MODEL_CLASSES

    async def classify(self, video_data):
        self.logger.info(f'Model IN use : {self.image_classification_model}')
        label = await self.video_model.predict(classifier_model_name={"model": self.image_classification_model, "classes": self.image_classification_model_classes}, video=video_data)
        return label

In [1]:
video_classification_service = VideoClassificationService()
# 
video = await BasicVideoUtils.read_video_file(content, filename=file.filename, cache=True)
video_category = await video_classification_service.classify(video)

NameError: name 'BasicVideoUtils' is not defined